In [5]:
from PIL import Image
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import sys
import os
from multiprocessing import Pool
from functools import partial
directory = 'figures'
if not os.path.exists(directory):
    os.makedirs(directory)
from concurrent.futures import ProcessPoolExecutor, as_completed
from dask.distributed import Client, progress
from dask import compute, delayed
import dask.array as da
from dask.diagnostics import ProgressBar
plt.rcParams['figure.dpi']=400

# Problem 4.1

In [ ]:
def load_image(num):
    # loads image, converts to grayscale, then converts to a matrix
    image = Image.open('images/'+f'image{num}.png')
    gray_image = image.convert('L')
    matrix = np.array(gray_image)
    return matrix

def compute_svd(image_matrix):
    # computes the singular value decomposition of a matrix
    U, s, Vt = np.linalg.svd(image_matrix, full_matrices=False)
    return U, s, Vt

def compress_image(U, s, Vt, k):
    # compresses it to rank k
    S = np.diag(s[:k])
    return np.dot(U[:, :k], np.dot(S, Vt[:k, :]))

def frobenius_percent(original, approx):
    # computes frobenius error
    return 100*(np.linalg.norm(original - approx, 'fro')/np.linalg.norm(original, 'fro'))

def memory_saved(m, n, k, s):
    # computes percentage memory saved
    new = (m * k + k + n * k)
    old = m * len(s) + len(s) + n * len(s)
    return 100*(old-new)/old
    

In [ ]:
path = 2
A = load_image(path)
U, s, Vt = compute_svd(A)
N = int(np.log2(len(s)))
ks=[]

for i in range(2, N+1):
    ks.append(len(s)//2**i) 
    
images = [A]
titles = [f'Original, k = {len(s)}']
frobenius=[0]
m, n = A.shape
memory=[0]

for k in ks:
    A_k = compress_image(U, s, Vt, k)
    images.append(A_k)
    frobenius.append(frobenius_percent(A, A_k))
    memory.append(memory_saved(m, n, k, s))
    titles.append(f'k = {k}')

In [ ]:
num_images = len(images)
cols = min(num_images, 3)
rows = num_images // cols + (num_images % cols > 0)

fig, axs = plt.subplots(rows, cols, figsize=(cols * (4*(m/n)), rows * 1.25*(n/m)))
fig.subplots_adjust(hspace=0, wspace=0)

axs = axs.ravel() if num_images > 1 else [axs]

for i in range(num_images):
    axs[i].imshow(images[i], cmap='gray')
    axs[i].set_title(f'{titles[i]}\nFrobenius error: {frobenius[i]:.3g}%\nMemory saved: {memory[i]:.3g}%', fontsize=5)
    axs[i].axis('off')
for ax in axs[num_images:]:
    ax.axis('off')
plt.savefig(os.path.join(directory, 'compressed-milkyway.png'), dpi=400)
plt.show()

# Problem 4.2

In [6]:
def sparseH(L, J, h, periodic):
    
    """
    generates the sparse Hamiltonian matrix for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (csr_matrix): sparse matrix representing the Hamiltonian operator
    """
    
    dim = 2 ** L # dimensions of the Hilbert space
    
    # initialize 
    H_data = []
    H_rows = []
    H_cols = []
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1, L + 1): # iterate over all sites
            
            alpha = beta ^ (1 << (j - 1)) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            "Keep track of the indices with non-zero matrix elements"
            
            H_data.append(-h)
            H_rows.append(alpha)
            H_cols.append(beta)
    
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim):  # iterate over all states
        
        A = 0
        
        for j in range(1, L): # iterate over all sites
            
            if 2 * (alpha & (1 << (j - 1))) == alpha & (1 << j): # check if site j and j+1 have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
                
        "Handling periodic boundary conditions"
                
        if periodic and L > 1: # L > 1 needed for periodicity to mean anything
            
            if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))): # Check if the states at either end have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
        
        if A != 0: # Check if the resulting matrix element is non-zero, if so, keep track of it
        
            H_data.append(A)
            H_rows.append(alpha)
            H_cols.append(alpha)

    H_data = np.array(H_data, dtype=float) # convert the list into a np array
    
    H = scipy.sparse.csr_matrix((H_data, (H_rows, H_cols)), shape=(dim, dim), dtype=np.float64) # make it into a csr sparse matrix
    
    return H

In [11]:
def psi_gs(L, J, h, periodic):
    H = sparseH(L, J, h, periodic)
    return scipy.sparse.linalg.eigsh(H, k=1, which='SA')[1]

In [76]:
def coefficient_matrix(l, L, vec):
    
    matrix = np.zeros((2**l, 2**(L-l)))
    
    for alpha in range(2**L):
        
        beta = alpha >> (L-l)
        
        gamma = alpha - (beta << (L-l))
        
        matrix[beta, gamma] = vec[alpha][0]
    
    return matrix
    
                      

In [99]:
def entropy(l, L, J, h, periodic):
    # computes S(l;L)
    
    vec = psi_gs(L, J, h, periodic)
    
    coeff_matrix = vec.reshape((2**l, 2**(L-l)))
    
    rho_A = np.zeros((2**l, 2**l), dtype=complex)

    for i in range(2**l):
        for k in range(2**(l)):
            # Sum over the states of A^c
            rho_A[i, k] = np.dot(coeff_matrix[i, :], coeff_matrix[k, :].conj())


    eigenvalues = np.linalg.eigh(rho_A)[0]

    S = -np.sum(eigenvalues * np.log(eigenvalues))
    
    return S

In [100]:
print(entropy(3, 10, 1, 1, False))
print(entanglement_entropy(3, 10, 1, 1, False))

0.35930701512653634
0.35930701512653573


In [94]:
def entanglement_entropy(l, L, J, h, periodic):
    """
    Calculate the entanglement entropy of a segment of the chain.

    Parameters:
        psi (numpy.ndarray): The ground state wavefunction.
        L (int): Total length of the chain.
        l (int): Length of the segment to consider for entanglement.

    Returns:
        S (float): The entanglement entropy of the segment.
    """
    vec = psi_gs(L, J, h, periodic)
    # Reshape psi into a matrix with dimensions corresponding to subsystems A and A^c
    psi_matrix = vec.reshape((2**l, 2**(L-l)))

    # Perform singular value decomposition
    u, s, vh = np.linalg.svd(psi_matrix, compute_uv=True)

    # Calculate the squares of the singular values, which are the eigenvalues of rho_A
    lambdas = s**2
    # Calculate the entanglement entropy
    S = -np.sum(lambdas * np.log(lambdas))

    return S

In [39]:
print(entanglement_entropy(3, 10, 1, 1, False))

[[0.29339911]
 [0.14835658]
 [0.09318807]
 ...
 [0.09318807]
 [0.14835658]
 [0.29339911]]
[[0.29339911 0.14835658 0.09318807 ... 0.03252672 0.05120963 0.09941022]
 [0.08638464 0.04388784 0.02764973 ... 0.03676075 0.05825056 0.11426809]
 [0.09318807 0.04729094 0.02976789 ... 0.01636846 0.02590641 0.0507405 ]
 ...
 [0.0507405  0.02590641 0.01636846 ... 0.02976789 0.04729094 0.09318807]
 [0.11426809 0.05825056 0.03676075 ... 0.02764973 0.04388784 0.08638464]
 [0.09941022 0.05120963 0.03252672 ... 0.09318807 0.14835658 0.29339911]]
[8.85364722e-01 1.14167317e-01 4.14376315e-04 5.34336088e-05
 1.33550255e-07 1.72212354e-08 6.25053846e-11 8.06003661e-12]
0.35930701512653634
